In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

# *Deep Reinforced Learning*

Hoy en día el estado del arte en el reconocimiento de patrones está dominado por las **redes neuronales profundas**

- Visión Computacional: Redes Neuronales Convolucionales, Adversarios generativos
- Reconocimiento de habla: Redes Recurrentes, WaveNet
- Procesamiento de lenguaje Natural: Transformers

Las redes neuronales son excelentes para representar el mundo: modelos

> Podemos aprovechar esta capacidad para diseñar mejores algoritmos de aprendizaje reforzado

# Recuerdo: Value Learning

En este tipo de algoritmos usamos un política de máxima utilidad para escoger acciones

$$
\pi^*(s) = \text{arg} \max_{a\in \mathcal{A}} Q(s, a)
$$

Y el problema entonces se reduce a aprender **Q**, *e.g* Q-Learning

Sin embargo Q-learning tiene limitaciones: 
- requiere de heurísticas para explorar 
- espacio de acciones debe ser discreto
- espacio de estados debe ser discreto

De hecho el estado no puede ser demasiado grande, como veremos a continuación

#### Ambiente: [Space invaders](https://es.wikipedia.org/wiki/Space_Invaders)

Originalmente un juego de Arcade lanzando en 1978, en 1980 tuvo una versión para ATARI 2600

> El objetivo es derrivar a los extraterrestres usando un cañon antes de que desciendan a la Tierra

- El cañon puede moverse a la izquierda, a la derecha y disparar
- Hay cuatro escudos con los que el cañon puede protegerse de los disparos enemigos
- Mientras menos enemigos más rápido se mueven

In [ ]:
import gym
from time import sleep

env = gym.make("SpaceInvaders-v0") 
env.reset()
end = False

while not end:
    a = env.action_space.sample()
    s, r, end, info = env.step(a)
    env.render() 
    sleep(.02)     

In [ ]:
env.close()

In [ ]:
# Espacio de estados
display(env.observation_space)
# Espacio de acciones
display(env.action_space)
display(env.action_space.n)
display(env.env.get_action_meanings())

Por ejemplo si usamos como estado un stack de 4 imágenes consecutivas y asumimos que cada pixel tiene 255 niveles:

255**(210*160*3*4) 

# Aproximación de funciones

Claramente, el espacio de estados del ejemplo anterior es imposible de mantener en una tabla Q

Ojo: este espacio de estados está aun lejos de un problema del mundo real

> Nos va a faltar memoría para guardar la tabla y datos para poder entrenar nuestro agente

¿Qué podemos hacer?

> Usar una representación más compacta para Q

En lugar de tener una tabla con todos las combinaciones estado/acción podemos

> Aproximar **Q** usando un **modelo paramétrico**

Esta es la idea principal tras *Value function approximation* (VFA) y *Q function approximation*

El caso más sencillo es usar un **modelo lineal en sus parámetros**

$$
\begin{align}
\hat Q_\theta(s,a) &= \theta_0 \phi_0(s, a) + \theta_1 \phi_1(s, a) + \theta_2 \phi_2(s, a) + \ldots + \theta_M \phi_M (s,a) \nonumber \\
&= \sum_{j=0}^M \theta_j \phi_j (s,a) \nonumber
\end{align}
$$

donde 
- $\{ \theta\}$ es un vector de parámetros con $M+1$ componentes
- $\{\phi\}$ es un conjunto de funciones base, *e.g.* polinomios, Fourier, árbol de decisión, kernels

En lugar de aprender $Q$ explicitamente el objetivo es aprender $\theta$

> La cantidad de parámetros es ahora independiente de la dimensionalidad del estado



Al igual que antes nuestro objetivo es acercanos a la solución de la Ecuación de Bellman

Podemos escribir esto como el siguiente problema de optimización

$$
\min_\theta \| R(s,a) + \gamma \max_{a' \in \mathcal{A}} \hat Q_\theta( s',a') - \hat Q_\theta(s,a)\|^2
$$

de donde podemos aprender $\theta$ iterativamente usando usando gradiente descendente 

$$
\theta_j \leftarrow \theta_j + 2 \alpha \left(R(s,a) + \gamma \max_{a' \in \mathcal{A}} \hat Q_\theta (s',a') - \hat Q_\theta(s,a) \right) \phi_j(s,a)
$$

Sin embargo, un modelo lineal podría ser muy limitado

En la unidad 1 estudiamos el estado del arte en aproximación de funciones: **redes neuronales artificiales**

> A continuación veremos como usar redes profundas para aproximar la función Q

# Deep Q-Network (DQN) 

> En [(Minh et al. 2013)](https://arxiv.org/abs/1312.5602) se usaron redes neuronales profundas de tipo convolucional para resolver una serie de juegos de ATARI con Aprendizaje Reforzardo obteniendo [desempeño sobre-humano en muchos de las pruebas](https://deepmind.com/blog/article/deep-reinforcement-learning). El modelo, llamado *Deep Q-network*, utiliza como estado el valor de todos los píxeles de cuatro cuadros consecutivos.

La idea clave es

> Aprovechar la capacidad de las redes neuronales profundas para representar datos complejos, e.g. imágenes 

o más en concreto

> Aproximar la función Q usando una red convolucional entrenada directamente sobre los píxeles

Veremos primero una formulación general y luego su aplicación al caso de imágenes

### Modelo DQN

- La entrada de la red es el estado $s$. 
    - El vector de estado puede tener valores continuos o discretos
- La salida de la red neuronal son los valores $Q(s, a_1), Q(s, a_2), \ldots, Q(s, a_N)$
    - Se considera un espacio de acciones discreto
    - Esto es más eficiente que considerar $a'$ como una entrada y retornar $Q(s,a')$
- La cantidad y tipo de las capas intermedias es decisión del usuario
    - Si tenemos datos continuos (atributos) usamos capas completamente conectadas
    - Si usamos píxeles es natural usar capas convolucionales
- La función de perdida que se ocupa en DQN es el error cuadrático medio entre la ecuación de Bellman y la predicción de la red

$$
\mathcal{L}(\theta) = \mathbb{E}\left[\left \| R(s,a) + \gamma \max_{a' \in \mathcal{A}} Q_\theta(s', a') - Q_\theta(s, a)\right \|^2\right]
$$

Recordemos: $s'$ es el estado al que llegamos luego de ejecutar $a$ sobre $s$

# DQN con estados continuos: El retorno del carro con péndulo

Previamente fue necesario discretizar el estado para construir la matriz Q

En DQN podemos obviar este paso y sus complicaciones

Vamos a usar una red neuronal con 3 capas completamente conectadas, 4 entradas y 2 salidas

In [ ]:
import torch

class DQN_FC(torch.nn.Module):    
    def __init__(self, n_input, n_output, n_hidden=10):
        super(DQN_FC, self).__init__()
        self.linear1 = torch.nn.Linear(n_input, n_hidden)
        self.linear2 = torch.nn.Linear(n_hidden, n_hidden)
        self.linear3 = torch.nn.Linear(n_hidden, n_output)
        self.activation = torch.nn.ReLU()
        
    def forward(self, x):
        h = self.activation(self.linear1(x))
        h = self.activation(self.linear2(h))
        return  self.linear3(h)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots(3, figsize=(6, 5), sharex=True, tight_layout=True)

def update_plot():
    for ax_ in ax:
        ax_.cla()
    
    episodes = np.arange((episode+1)//100)*100
    ax[0].errorbar(episodes,
                   np.array(diagnostics['rewards']).mean(axis=1), 
                   np.array(diagnostics['rewards']).std(axis=1));
    ax[0].set_ylabel('Recompensa\npromedio');
    ax[1].errorbar(episodes,
                   np.array(diagnostics['episode_length']).mean(axis=1), 
                   np.array(diagnostics['episode_length']).std(axis=1));
    ax[1].plot(episodes, [195]*len(episodes), 'k--')
    ax[1].set_ylabel('Largo promedio\nde los episodios');
    ax[2].plot(episodes, epsilon(episodes))
    ax[2].set_ylabel('Epsilon')
    ax[2].set_xlabel('Episodios')
    fig.canvas.draw()

#### Consideraciones

- El optimizador a usar es ADAM: Gradiente descendente con tasa de aprendizaje adaptiva
- Se usa la heurística $\epsilon$ greedy para favorecer la exploración

In [ ]:
import numpy as np
import gym
from tqdm.notebook import tqdm

env = gym.make("CartPole-v0")

model_policy = DQN_pole(n_hidden=10) 
double_dqn = False
if double_dqn:
    model_target = DQN_pole(n_hidden=10)
criterion = torch.nn.MSELoss()
# criterion = torch.nn.SmoothL1Loss() # Huber Loss
optimizer = torch.optim.Adam(model_policy.parameters(), lr=1e-3)

diagnostics = {'rewards': [], 'episode_length': []}
# Parametros
gamma = 0.999
epsilon_init = 1.0 
epsilon_end = 0.01 
epsilon_rate = 1e-3
epsilon = lambda episode : epsilon_end + (epsilon_init - epsilon_end) * np.exp(-epsilon_rate*episode) 

for episode in tqdm(range(3000)):
    env.reset()
    end = False
    # Entrenamiento
    while not end:        
        # Seleccionar la acción        
        s_current = torch.from_numpy(np.array(env.state).astype('float32')) # NUEVO
        pred = model_policy(s_current) # NUEVO
        if not np.random.binomial(1, p=1.-epsilon(episode)): 
            Q_present = pred.max() # NUEVO
            a = pred.argmax().item() # NUEVO  
        else:                        
            a = env.action_space.sample() 
            Q_present = pred[a]
        # Ejecutar la acción
        s, r, end, info = env.step(a)
        # Actualizar la red Q
        s_future = torch.from_numpy(np.array(s).astype('float32')) # NUEVO
        
        if double_dqn:
            Q_future = model_target.predict(s_future)
        else:
            Q_future = model_policy(s_future)
        
        target = torch.tensor(r)
        if not end:
            target += gamma*Q_future.max().detach()
        
        loss = criterion(target, Q_present)
        optimizer.zero_grad() # NUEVO
        loss.backward() # NUEVO
        optimizer.step() # NUEVO
                
        if double_dqn:
            if episode % 10 == 0:
                model_target.load_state_dict(model_policy.state_dict())

    # Prueba
    # Cada 100 epocas evaluamos nuestro agente
    if np.mod(episode+1, 100) == 0:
        diagnostics['rewards'].append(np.zeros(shape=(10,)))
        diagnostics['episode_length'].append(np.zeros(shape=(10,)))
        for k in range(10):
            env.reset()    
            end = False
            episode_length = 0
            episode_reward = 0.0
            while not end:        
                s_current = torch.from_numpy(np.array(env.state).astype('float32')) 
                a = model_policy(s_current).argmax().detach().numpy() 
                s_future, r, end, info = env.step(a)
                episode_length += 1
                episode_reward += r
            
            diagnostics['rewards'][-1][k] = episode_reward
            diagnostics['episode_length'][-1][k] = episode_length
        update_plot()

Nuevamente, nuestro agente en acción

In [ ]:
import numpy as np
import gym

env = gym.make("CartPole-v0")

In [ ]:
env.reset()
end = False

for k in range(500):
    env.render()
    s_current = torch.from_numpy(np.array(env.state).astype('float32'))
    a = model_policy(s_current).argmax().detach().numpy() 
    s_future, r, end, info = env.step(a)
    #if end:
    #    break
    if r == 0:
        display(k)
        break
display(end, k)

In [ ]:
env.close()

# Experience Replay y Memory replay

El entrenamiento usando sólo una instancia es bastante ruidoso (y un poco lento)

Consideremos también que estamos entrenando con muestras muy correlacionadas (no iid)

Sabemos que esto puede introducir sesgos el entrenamiento de una red neuronal

> Para entrenar la DQN adecuadamente  (Minh et al 2013) propone un astuto "truco" llamado *Experience replay*  

Esto consiste en almacenar la historia del agente en una memoria: *replay memory*

Cada elemento en la memoria es una tupla $(s_t, a_t, r_{t+1}, s_{t+1})$

Con esto se crean mini-batches en orden aleatorio para entrenar



In [ ]:
class ReplayMemory:
    
    def __init__(self, state_dim, memory_length=1024):
        self.length = memory_length
        self.pointer = 0
        self.filled = False
        # Tensores vacíos para la historia
        self.s_current = torch.zeros(memory_length, state_dim)
        self.s_future = torch.zeros(memory_length, state_dim)
        self.a = torch.zeros(memory_length, 1, dtype=int)
        self.r = torch.zeros(memory_length, 1)
        # Adicionalmente guardaremos la condición de término
        self.end = torch.zeros(memory_length, 1, dtype=bool)
    
    def push(self, s_current, s_future, a, r, end):
        # Agregamos una tupla en la memoria
        self.s_current[self.pointer] = s_current
        self.s_future[self.pointer] = s_future
        self.a[self.pointer] = a
        self.r[self.pointer] = r 
        self.end[self.pointer] = end
        if self.pointer + 1 == self.length:
            self.filled = True
        self.pointer =  (self.pointer + 1) % self.length    
        
    def sample(self, size=128):
        # Extraemos una muestra aleatoria de la memoria
        idx = np.random.choice(self.length, size)
        return self.s_current[idx], self.s_future[idx], self.a[idx], self.r[idx], self.end[idx]
    

# *Double DQN*


En [(van Hasselt, Guer y Silver, 2015)](https://arxiv.org/pdf/1509.06461.pdf) los autores notaron un problema importante en DQN. 

> Cuando calculamos $Q(s, a)$ y $\max Q(s', a')$ usando la misma red es muy posible que sobre-estimemos la calidad de nuestro objetivo. Adicionalmente si el objetivo cambia constantemente el entrenamiento será inestable


La solución propuesta consiste en usar redes neuronales distintas para la escoger la acción y para calcular el objetivo (ecuación de Bellman)

$$
\mathcal{L}(\theta, \phi) = \mathbb{E}\left[\left \| R(s,a) + \gamma \max_{a' \in \mathcal{A}} Q_\phi(s', a') - Q_\theta(s, a)\right\|^2\right]
$$

Usamos $Q_\theta$ con parámetros $\theta$ para escoger la acción: *policy network*

Usamos $Q_\phi$ con parámetros $\phi$ para construir el objetivo: *target network

- Ambas redes comparten arquitectura y cantidad de neuronas
- Sólo se optimiza la *policy network*
- Después de un cierto número de épocas los parametros de la policy network "se copian" en la *target network*

¿Cada cuantas épocas actualizo la *target network*? Otro hyperparámetro para el algoritmo... 

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.signal import convolve

fig, ax = plt.subplots(3, figsize=(6, 5), sharex=True, tight_layout=True)

def update_plot(episode, smooth_window=10):
    for ax_ in ax:
        ax_.cla()
    episodes = np.arange((episode))
    if episode > smooth_window:
        ax[0].plot(episodes[:-smooth_window+1], 
                   convolve(diagnostics['rewards'][:episode], 
                            np.ones(smooth_window), mode='valid')/smooth_window)        
        ax[1].plot(episodes[:-smooth_window+1], 
                   convolve(diagnostics['loss'][:episode], 
                            np.ones(smooth_window), mode='valid')/smooth_window)
    ax[0].plot(episodes, [195]*len(episodes), 'k--')
    ax[0].set_ylabel('Recompensa\npromedio');
    ax[1].set_ylabel('Loss')
    ax[2].plot(episodes, epsilon(episodes))
    ax[2].set_ylabel('Epsilon')
    ax[2].set_xlabel('Episodios')
    fig.canvas.draw()

In [ ]:
import numpy as np
import gym
from tqdm.notebook import tqdm

env = gym.make("CartPole-v0")

n_state = env.observation_space.shape[0] # Número de estados
n_action = env.action_space.n # Número de acciones

model_policy = DQN_FC(n_state, n_action, n_hidden=100) 

double_dqn = False
if double_dqn:
    model_target = DQN_FC(n_state, n_action, n_hidden=100)
    
criterion = torch.nn.MSELoss() # Error medio cuadrático
#criterion = torch.nn.SmoothL1Loss() # Huber Loss
# Solo optimizaremos la policy network
optimizer = torch.optim.Adam(model_policy.parameters(), lr=1e-3)


gamma = 0.999
def epsilon(episode, epsilon_init=1.0, epsilon_end=0.01, epsilon_rate=1e-3):
    return epsilon_end + (epsilon_init - epsilon_end) * np.exp(-epsilon_rate*episode) 

memory = ReplayMemory(n_state)        
num_episodes = 3000
diagnostics = {'rewards': np.zeros(shape=(3000,)), 
               'loss': np.zeros(shape=(3000,))}

for episode in tqdm(range(num_episodes)):
    env.reset()
    end = False
    episode_reward = 0
    episode_loss = 0.0
    while not end:        
        # Llenar memoria
        s_current = torch.tensor(env.state).float()
        if not np.random.binomial(1, p=1.-epsilon(episode)): 
            with torch.no_grad():
                a = model_policy(s_current).argmax().item() # NUEVO  
        else:                        
            a = env.action_space.sample() 
        s, r, end, info = env.step(a)
        episode_reward += r
        memory.push(s_current, torch.tensor(s).float(), a, r, end)
        
        # Entrenamiento
        if memory.filled:
            state_torch, state_future_torch, action_torch, reward_torch, end_torch = memory.sample()
            # Obtener el valor de Q del estado actual y de la acción actual
            prediction = model_policy(state_torch).gather(1, action_torch)
            # Obtener el valor del mejor Q del estado futuro
            if not double_dqn:
                Q_future_best = model_policy(state_future_torch).max(1, keepdim=True)[0].detach()
            else:
                Q_future_best = model_target(state_future_torch).max(1, keepdim=True)[0].detach()
            # Construir el target: r + gamma*max Q(s', a')
            target = reward_torch
            target[~end_torch] += gamma*Q_future_best[~end_torch]
            # Actualizar
            optimizer.zero_grad()
            loss = criterion(prediction, target)
            loss.backward()
            #for param in model_policy.parameters():
            #    param.grad.data.clamp_(-1, 1)
            optimizer.step()
            episode_loss += loss.item()
    
    diagnostics['rewards'][episode] = episode_reward 
    diagnostics['loss'][episode] = episode_loss 
    
    if double_dqn:
        if episode % 10 == 0:
            model_target.load_state_dict(model_policy.state_dict())
                
    if episode % 50 == 0:
        update_plot(episode)